# F-тест: значимость регрессии

In [ ]:
import pandas as pd
import numpy as np

from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col # вывод результатов тестирования

from scipy.stats import f # f-распределение и критические значения

# Не показывать Warning
import warnings
warnings.simplefilter(action='ignore', category=Warning)

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

<i><b><span style="color: purple">Значимость регрессии </span> </b><br>

Для линейной регрессии

$$
	(\log)y=\beta_0+\beta_1(\log)x_1+\cdots+\beta_k(\log)x_k+u=x^\top \beta+u
$$

Тестируется гипотеза

$$
	H_0:\beta_1=\cdots=\beta_k=0
$$

__Интерпретация__: включённые в модель регрессоры совместно не влияют на зависимую переменную (в генеральной совокупности)

__Тестовая статистика__ для выбранной оценки ковариационной матрицы $\hat{V}$ формула для F-статистики (всё вычисляется автоматически!)

$$
\begin{aligned}
	F&=\frac{1}{k}(R\widehat{\beta})^\top
	\Bigl(R\cdot\hat{V}\cdot R^\top\Bigr)^{-1}
	(R\widehat{\beta}) &
	\underset{k\times (k+1)}{R}&=
	\begin{pmatrix} 
	0 & 1 & 0 & \cdots & 0 \\ 
	0 & 0 & 1 & \cdots & 0 \\
	\vdots & \vdots & \vdots & \ddots & \vdots \\
	0 & 0 & 0 & \cdots & 1
	\end{pmatrix}=\begin{pmatrix} 0_{k\times 1} & I_{k\times k}\end{pmatrix}
\end{aligned}
$$

__Критическое значение__: $F$-распределения со степенями свободы ($k$ - число объясняющих переменных)

$$ 
\begin{aligned} 
	df_1&=dfn=k & df_2&=dfd=n-k-1
\end{aligned}
$$

__Замечание__: 
* для F-тест есть неробастный (по умолчанию) и робастный вариант
* для специфицированной модели

<div style="margin-left: auto;
            margin-right: auto;
            width: 30%">
			
|Свойство|Значение|
|-|-|
|`.df_model` |$k$|
|`.df_resid`| $n-k-1$|

</div>

* для подогнанной регрессии 

<div style="margin-left: auto;
            margin-right: auto;
            width: 30%">

|Свойство|Значение|
|-|-|
|`.df_model` |$k$|
|`.df_resid`| $n-k-1$|
|`.fvalue`|F-статистика|
|`.f_pvalue`|P-значение для F|

</div>

__Вывод__

**<span style="color:purple">Способ 1:</span>** используем $F$-статистику

* Отвергаем гипотезу $H_0$ при $F>F_{cr}$, <i><b><span style="color: purple">регрессия значима</span></b><br>
* Не отвергаем гипотезу $H_0$ при $F<F_{cr}$, <i><b><span style="color: purple">регрессия незначима</span> </b><br>

**<span style="color:purple">Способ 2:</span>** используем $P$-value

* Отвергаем гипотезу $H_0$  при $P<\alpha$, <i><b><span style="color: purple">регрессия значима</span> </b><br>
* Не отвергаем гипотезу $H_0$ при $P>\alpha$, <i><b><span style="color: purple">регрессия незначима</span> </b><br>

</div>

Для набора данных `sleep75` рассмотрим линейную регрессию __sleep~smsa+yngkid+marr+union__

Значимость регрессии: тестируем гипотезу

$$
	H_0:\beta_{smsa}=\beta_{yngkid}=\beta_{marr}=\beta_{union}=0
$$



In [ ]:
# импорт данных
df = pd.read_csv('sleep75.csv')

## Значимость регрессии (неробастный вариант)

Используем OLS-оценку ковариационной матрицы $\hat{V}=s^2(X^\top X)^{-1}$

В это случаем F-статистика вычисляется по формуле

$$
	F=\frac{R^2}{1-R^2}\cdot\frac{n-k-1}{k}=\frac{Var(\hat{y})}{Var(e)}\cdot\frac{n-k-1}{k}=
	\frac{ESS}{RSS}\cdot\frac{n-k-1}{k}
$$

In [ ]:
# спецификация модели через формулу
mod = ols(formula='sleep~1+smsa+yngkid+marr+union', data=df)
# подгонка модели с неробастной оценкой ковариационной матрицы
res_ols = mod.fit()

In [ ]:
# F-статистика и её P-значение
res_ols.fvalue, res_ols.f_pvalue

In [ ]:
# уровень значимости
sign_level= 0.05
# критическое значение
f.isf(q=sign_level, dfn=mod.df_model, dfd=mod.df_resid)

In [ ]:
# степени свободы для F-распределения
mod.df_model, mod.df_resid

При выведении отчёта о подгонке модели с неробастной (OLS-) оценкой ковариационной матрицы __неробастная тестовая F-статистика включается в итоговый протокол__

In [ ]:
res_ols.summary(slim=True)

## Вывод

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

Не отвергаем гипотезу $H_0$ (т.к. $F<F_{cr}$ или $P>\alpha$), <i><b><span style="color: purple">регрессия незначима</span></b><br>

</div>

## Значимость регрессии (робастный тест)

При подгонке используем HC3-оценку ковариационной матрицы 

In [ ]:
# спецификация модели через формулу
mod = ols(formula='sleep~1+smsa+yngkid+marr+union', data=df)
# подгонка модели с робастной HC3-оценкой ковариационной матрицы
res_hc = mod.fit(cov_type='HC3')

In [ ]:
# робастная F-статистика и её P-значение
res_hc.fvalue, res_hc.f_pvalue

In [ ]:
# уровень значимости
sign_level= 0.05
# критическое значение
f.isf(q=sign_level, dfn=mod.df_model, dfd=mod.df_resid)

При выведении отчёта о подгонке модели с робастной оценкой ковариационной матрицы __робастная тестовая F-статистика включается в итоговый протокол__

In [ ]:
res_hc.summary(slim=True)

## Вывод

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

Не отвергаем гипотезу $H_0$ (т.к. $F<F_{cr}$ или $P>\alpha$), <i><b><span style="color: purple">регрессия незначима</span></b><br>

</div>